In [35]:
%logstop
%logstart -rtq ~/.logs/dw.py append
import matplotlib
import seaborn as sns
import pandas as pd
import math
import gzip
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [3]:
from static_grader import grader

# DW Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to wrangle tabular data set and aggregate large data sets into meaningful summary statistics. We'll work with the same medical data used in the `pw` miniproject but leverage the power of Pandas to more efficiently represent and act on our data.

## Downloading the data

We first need to download the data we'll be using from Amazon S3:

In [4]:
!mkdir dw-data
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/201701scripts_sample.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/201606scripts_sample.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/practices.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/chem.csv.gz -nc -P ./dw-data/

mkdir: cannot create directory ‘dw-data’: File exists
File ‘./dw-data/201701scripts_sample.csv.gz’ already there; not retrieving.

File ‘./dw-data/201606scripts_sample.csv.gz’ already there; not retrieving.

File ‘./dw-data/practices.csv.gz’ already there; not retrieving.

File ‘./dw-data/chem.csv.gz’ already there; not retrieving.



## Loading the data

Similar to the `pw` miniproject, the first step is to read in the data. The data files are stored as compressed CSV files. You can load the data into a pandas `DataFrame` by making use of the `pd.read_csv` function to parse the data into a `DataFrame`. You may want to check the pandas documentation for parsing [CSV](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) files for reference.

For a description of the data set please, refer to the [`pw` miniproject](./pw.ipynb). **Note that all questions make use of the 2017 data only, except for Question 5 which makes use of both the 2017 and 2016 data.**

In [5]:
import pandas as pd
import numpy as np
import gzip

In [6]:
# load the 2017 data
scripts = pd.read_csv('./dw-data/201701scripts_sample.csv.gz')
scripts.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6


In [7]:
col_names=[ 'code', 'name', 'addr_1', 'addr_2', 'borough', 'village', 'post_code']

practices = pd.read_csv('./dw-data/practices.csv.gz')
practices.columns = col_names
practices.head()

,code,name,addr_1,addr_2,borough,village,post_code
0,A81002,QUEENS PARK MEDICAL CENTRE,QUEENS PARK MEDICAL CTR,FARRER STREET,STOCKTON ON TEES,CLEVELAND,TS18 2AW
1,A81003,VICTORIA MEDICAL PRACTICE,THE HEALTH CENTRE,VICTORIA ROAD,HARTLEPOOL,CLEVELAND,TS26 8DB
2,A81004,WOODLANDS ROAD SURGERY,6 WOODLANDS ROAD,NaN,MIDDLESBROUGH,CLEVELAND,TS1 3BE
3,A81005,SPRINGWOOD SURGERY,SPRINGWOOD SURGERY,RECTORY LANE,GUISBOROUGH,NaN,TS14 7DJ
4,A81006,TENNANT STREET MEDICAL PRACTICE,TENNANT ST MED PRACT,FARRER STREET,STOCKTON ON TEES,CLEVELAND,TS18 2AT


In [8]:
chem = pd.read_csv('./dw-data/chem.csv.gz')
col_names=[ 'chem sub', 'name']
chem.columns = col_names
chem.head()

,chem sub,name
0,0101010A0,Alexitol Sodium
1,0101010B0,Almasilate
2,0101010C0,Aluminium Hydroxide
3,0101010D0,Aluminium Hydroxide With Magnesium
4,0101010E0,Hydrotalcite


Now that we've loaded in the data, let's first replicate our results from the `pw` miniproject. Note that we are now working with a larger data set so the answers will be different than in the `pw` miniproject even if the analysis is the same.

## Question 1: summary_statistics

In the `pw` miniproject we first calculated the total, mean, standard deviation, and quartile statistics of the `'items'`, `'quantity'`', `'nic'`, and `'act_cost'` fields. To do this we had to write some functions to calculate the statistics and apply the functions to our data structure. The DataFrame has a `describe` method that will calculate most (not all) of these things for us.

Submit the summary statistics to the grader as a list of tuples: `[('act_cost', (total, mean, std, q25, median, q75)), ...]`

In [12]:
summary_stats = [('items', (0,) * 6), ('quantity', (0,) * 6), ('nic', (0,) * 6), ('act_cost', (0,) * 6)]

In [13]:
def summary(key):
    total = scripts[key].sum()
    avg = scripts[key].describe()[1]
    s = scripts[key].describe()[2]
    q25 = scripts[key].describe()[4]
    med = scripts[key].median()
    q75 = scripts[key].describe()[6]
    return (total, avg, s, q25, med, q75)

In [14]:
summary_stats = [('items', summary('items')), 
                 ('quantity', summary('quantity')), 
                 ('nic', summary('nic')), 
                 ('act_cost', summary('act_cost'))]

In [15]:
grader.score.dw__summary_statistics(summary_stats)

Your score: 1.000


## Question 2: most_common_item

We can also easily compute summary statistics on groups within the data. In the `pw` miniproject we had to explicitly construct the groups based on the values of a particular field. Pandas will handle that for us via the `groupby` method. This process is [detailed in the Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/groupby.html).

Use `groupby` to calculate the total number of items dispensed for each `'bnf_name'`. Find the item with the highest total and return the result as `[(bnf_name, total)]`.

In [16]:
most_common_item = [("", 0)]

In [20]:
items = scripts.groupby(['bnf_name'], sort=False)['items'].sum()
items_df = pd.DataFrame({'bnf_name':items.index, 'items':items.values})
a = items_df.loc[items_df['items'].idxmax()]
most_common_item= list()
emptytuple= tuple()
emptytuple=a[0],a[1]
most_common_item.append(emptytuple)
print(most_common_item)

[('Omeprazole_Cap E/C 20mg', 218583)]


In [20]:
grader.score.dw__most_common_item(most_common_item)

Your score: 1.000


## Question 3: items_by_region

Now let's find the most common item by post code. The post code information is in the `practices` DataFrame, and we'll need to `merge` it into the `scripts` DataFrame. Pandas provides [extensive documentation](https://pandas.pydata.org/pandas-docs/stable/merging.html) with diagrammed examples on different methods and approaches for joining data. The `merge` method is only one of many possible options.

Return your results as a list of tuples `(post code, item name, amount dispensed as % of total)`. Sort your results ascending alphabetically by post code and take only results from the first 100 post codes.

**NOTE:** Some practices have multiple postal codes associated with them. Use the alphabetically first postal code. Note some postal codes may have multiple `'bnf_name'` with the same prescription rate for the maximum. In this case, take the alphabetically first `'bnf_name'` (as in the `pw` miniproject).

In [9]:
scripts.sort_values('bnf_name')
uniquepractice = practices.sort_values('post_code').groupby('code').first().reset_index()
joined = pd.merge(scripts, uniquepractice, how = 'inner', left_on = 'practice', right_on = 'code')

groups=joined.groupby(['post_code', 'bnf_name'])
sms=groups.sum()

g = sms['items'].groupby(level=0, group_keys=False)
res = g.nlargest(1)
res.head()

groups_post=joined.groupby(['post_code'])['items'].sum()

portion_of_total=0
lis=list()
final_list_grader=list()

count=0
for i, j in res.iteritems():
    if count<100:
        for ii, jj in groups_post.iteritems():
            if ii==i[0]:
                portion_of_total=j/jj
        emptytuple=tuple()

        emptytuple=i[0],i[1],round(portion_of_total,10)
        lis.append(emptytuple)
    count+=1
final_list_grader = lis

In [10]:
emptytuple

('L31 0DJ', 'Omeprazole_Cap E/C 20mg', 0.0294256537)

In [22]:
items_by_region = final_list_grader

In [23]:
grader.score.dw__items_by_region(items_by_region)

Your score: 1.000


## Question 4: script_anomalies

Drug abuse is a source of human and monetary costs in health care. A first step in identifying practitioners that enable drug abuse is to look for practices where commonly abused drugs are prescribed unusually often. Let's try to find practices that prescribe an unusually high amount of opioids. The opioids we'll look for are given in the list below.

In [12]:
opioids = ['morphine', 'oxycodone', 'methadone', 'fentanyl', 'pethidine', 'buprenorphine', 'propoxyphene', 'codeine']

These are generic names for drugs, not brand names. Generic drug names can be found using the `'bnf_code'` field in `scripts` along with the `chem` table. Use the list of opioids provided above along with these fields to make a new field in the `scripts` data that flags whether the row corresponds with a opioid prescription.

Now for each practice calculate the proportion of its prescriptions containing opioids.

**Hint:** Consider the following list: `[0, 1, 1, 0, 0, 0]`. What proportion of the entries are 1s? What is the mean value?

In [25]:
opioids_per_practice = ...

How do these proportions compare to the overall opioid prescription rate? Subtract off the proportion of all prescriptions that are opioids from each practice's proportion.

In [26]:
relative_opioids_per_practice = ...

Now that we know the difference between each practice's opioid prescription rate and the overall rate, we can identify which practices prescribe opioids at above average or below average rates. However, are the differences from the overall rate important or just random deviations? In other words, are the differences from the overall rate big or small?

To answer this question we have to quantify the difference we would typically expect between a given practice's opioid prescription rate and the overall rate. This quantity is called the **standard error**, and is related to the **standard deviation**, $\sigma$. The standard error in this case is

$$ \frac{\sigma}{\sqrt{n}} $$

where $n$ is the number of prescriptions each practice made. Calculate the standard error for each practice. Then divide `relative_opioids_per_practice` by the standard errors. We'll call the final result `opioid_scores`.

In [27]:
standard_error_per_practice = ...
opioid_scores = ...

The quantity we have calculated in `opioid_scores` is called a **z-score**:

$$ \frac{\bar{X} - \mu}{\sqrt{\sigma^2/n}} $$

Here $\bar{X}$ corresponds with the proportion for each practice, $\mu$ corresponds with the proportion across all practices, $\sigma^2$ corresponds with the variance of the proportion across all practices, and $n$ is the number of prescriptions made by each practice. Notice $\bar{X}$ and $n$ will be different for each practice, while $\mu$ and $\sigma$ are determined across all prescriptions, and so are the same for every z-score. The z-score is a useful statistical tool used for hypothesis testing, finding outliers, and comparing data about different types of objects or events.

Now that we've calculated this statistic, take the 100 practices with the largest z-score. Return your result as a list of tuples in the form `(practice_code, practice_name, z-score, number_of_scripts)`. Sort your tuples by z-score in descending order. Note that some practice codes will correspond with multiple names. In this case, use the first match when sorting names alphabetically.

In [50]:
unique_practices = ...
anomalies = [("Y01852", "NATIONAL ENHANCED SERVICE", 11.6958178629, 7)] * 100

In [22]:
chem = pd.read_csv('./dw-data/chem.csv.gz', compression='gzip', header=0, sep=',', quotechar='"', error_bad_lines=False)
chem.head()
chem.columns

with gzip.open ( './dw-data/201701scripts_sample.csv.gz', 'rb' ) as f:
    scripts = pd.read_csv ( f )

with gzip.open ( './dw-data/practices.csv.gz', 'rb' ) as f:
    practices = pd.read_csv ( f )



practices.columns = ['code', 'name', 'addr_1', 'addr_2', 'borough', 'village', 'post_code'] 
practices = practices[['code', 'name']].sort_values (by = ['name'], ascending = True) 
practices = practices [~practices.duplicated(['code'])] 
opioids = ['morphine', 'oxycodone', 'methadone', 'fentanyl', 'pethidine', 'buprenorphine', 'propoxyphene', 'codeine'] 


check = '|'.join(opioids) 
chem_df1 = chem 
chem_df1 [ 'test' ] = chem_df1 [ 'NAME' ].apply ( lambda x: any ( [ k in x.lower() for k in opioids ] ) ) 
key2 = chem_df1 [ "test" ] == True 
chem_df1 = chem_df1 [ key2 ]  
chem_sub = list (chem_df1['CHEM SUB']) 


scripts['opioid'] = scripts [ 'bnf_code' ].apply(lambda x: 1 if x in chem_sub else 0)
std_devn = scripts.opioid.std ()
overall_rate = scripts.opioid.mean()

scripts = scripts.merge (practices, left_on = 'practice', right_on = 'code')
scripts['cnt'] = 0


opioids_per_practice = scripts.groupby ( [ 'practice', 'name' ], as_index = False ).agg ( { 'opioid': 'mean', 'cnt': 'count' } )
opioids_per_practice.drop_duplicates()

opioids_per_practice['opioid'] = opioids_per_practice ['opioid'] - overall_rate

opioids_per_practice['std_err'] = std_devn / opioids_per_practice['cnt'] ** 0.5
opioids_per_practice['z_score'] = opioids_per_practice['opioid'] / opioids_per_practice['std_err']

result = opioids_per_practice[["practice", "name", "z_score", "cnt"]]


result = result.sort_values(by = 'z_score', ascending = False)

In [23]:
result.head()

,practice,name,z_score,cnt
714,Y01852,NATIONAL ENHANCED SERVICE,11.695818,7
754,Y03006,OUTREACH SERVICE NH / RH,7.339043,2
772,Y03668,BRISDOC HEALTHCARE SERVICES OOH,6.150582,60
261,G81703,H&R P C SPECIAL SCHEME,5.123032,36
819,Y04997,HMR BARDOC OOH,4.958866,321


In [29]:
anomalies = [(k[1], k[2], k[3], k[4]) for k in result.itertuples()][:100]

In [33]:
anomalies[:][:4]

[('Y01852', 'NATIONAL ENHANCED SERVICE', 11.695817862936027, 7),
 ('Y03006', 'OUTREACH SERVICE NH / RH', 7.339043019238823, 2),
 ('Y03668', 'BRISDOC HEALTHCARE SERVICES OOH', 6.1505817490838295, 60),
 ('G81703', 'H&R P C SPECIAL SCHEME', 5.123032414033079, 36)]

In [34]:
grader.score.dw__script_anomalies(anomalies)

Your score: 1.000


## Question 5: script_growth

Another way to identify anomalies is by comparing current data to historical data. In the case of identifying sites of drug abuse, we might compare a practice's current rate of opioid prescription to their rate 5 or 10 years ago. Unless the nature of the practice has changed, the profile of drugs they prescribe should be relatively stable. We might also want to identify trends through time for business reasons, identifying drugs that are gaining market share. That's what we'll do in this question.

We'll load in beneficiary data from 6 months earlier, June 2016, and calculate the growth rate in prescription rate from June 2016 to January 2017 for each `bnf_name`. The growth is defined as

$$
\text{growth rate} = \frac{x_1 - x_0}{x_0},
$$
where $x_1$ is the current value and $x_0$ is the previous value.


We'll return the 50 items with largest growth and the 50 items with the largest shrinkage (i.e. negative growth rate) as a list of tuples sorted by growth rate in descending order in the format `(script_name, growth_rate, raw_2016_count)`. You'll notice that many of the 50 fastest growing items have low counts of prescriptions in 2016. Filter out any items that were prescribed less than 50 times.

In [32]:
scripts16 = ...

In [10]:
script_growth = [("Butec_Transdermal Patch 5mcg\/hr", 3.4677419355, 62.0)] * 100

In [11]:
scripts16 = pd.read_csv('./dw-data/201606scripts_sample.csv.gz',compression='gzip', delimiter=',')
drugs_16 = scripts16[['bnf_name', 'items']]
drugs_16 = drugs_16.groupby('bnf_name').count().reset_index()
drugs_16.columns = ['bnf_name', 'count16']

drugs_17 = scripts[['bnf_name', 'items']]
drugs_17 = drugs_17.groupby('bnf_name').count().reset_index()
drugs_17.columns = ['bnf_name', 'count17']

drugs = drugs_16.merge(drugs_17, on='bnf_name', how='inner')
drugs = drugs[drugs['count16']>=50]

drugs.is_copy=False
drugs['growth'] = ((drugs['count17']-drugs['count16'])/drugs['count16'])
drugs = drugs[['bnf_name', 'growth', 'count16']]
drugs.sort_values('growth', ascending=False, inplace=True)
drugs_final = pd.concat([drugs.iloc[:50], drugs.iloc[-50:]], axis=0)

In [12]:
script_growth=drugs_final

In [13]:
grader.score.dw__script_growth(script_growth)

Your score: 1.000


## Question 6: rare_scripts

Does a practice's prescription costs originate from routine care or from reliance on rarely prescribed treatments? Commonplace treatments can carry lower costs than rare treatments because of efficiencies in large-scale production. While some specialist practices can't help but avoid prescribing rare medicines because there are no alternatives, some practices may be prescribing a unnecessary amount of brand-name products when generics are available. Let's identify practices whose costs disproportionately originate from rarely prescribed items.

First we have to identify which `'bnf_code'` are rare. To do this, find the probability $p$ of a prescription having a particular `'bnf_code'` if the `'bnf_code'` was randomly chosen from the unique options in the beneficiary data. We will call a `'bnf_code'` rare if it is prescribed at a rate less than $0.1p$.

In [14]:
p = ...
rates = ...
rare_codes = ...
scripts['rare'] = ...

Now for each practice, calculate the proportion of costs that originate from prescription of rare treatments (i.e. rare `'bnf_code'`). Use the `'act_cost'` field for this calculation.

In [15]:
rare_cost_prop = ...

Now we will calculate a z-score for each practice based on this proportion.
First take the difference of `rare_cost_prop` and the proportion of costs originating from rare treatments across all practices.

In [16]:
relative_rare_cost_prop = ...

Now we will estimate the standard errors (i.e. the denominator of the z-score) by simply taking the standard deviation of this difference.

In [17]:
standard_errors = ...

Finally compute the z-scores. Return the practices with the top 100 z-scores in the form `(practice_code, practice_name, z-score)`. Note that some practice codes will correspond with multiple names. In this case, use the first match when sorting names alphabetically.

In [18]:
rare_scores = ...

In [19]:
rare_scripts = [("Y03472", "CONSULTANT DIABETES TEAM", 16.2626871247)] * 100

In [20]:
rates = scripts['bnf_code'].value_counts() / scripts['bnf_name'].count()

rates.head()

0906040G0    0.014840
090402000    0.011016
0302000N0    0.010214
130201000    0.008839
0601060D0    0.008799
Name: bnf_code, dtype: float64

In [21]:
p = 1. /scripts['bnf_code'].nunique()

mask = rates < .1 * p

rare_codes = rates[mask].index

In [22]:
scripts['rare'] = scripts['bnf_code'].isin(rare_codes)

scripts.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity,rare
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12,False
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30,False
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1,False
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150,False
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6,False


In [23]:
rare_cost_prop = (scripts[scripts['rare']].groupby('practice')['act_cost'].sum()/ scripts.groupby('practice')['act_cost'].sum()).fillna(0)

rare_cost_prop.head()

practice
A81005    0.012017
A81007    0.008381
A81011    0.005116
A81012    0.013747
A81017    0.007359
Name: act_cost, dtype: float64

In [24]:
cost_all = scripts[scripts['rare']]['act_cost'].sum() / scripts['act_cost'].sum()

relative_rare_cost_prop = rare_cost_prop - cost_all

standard_errors = relative_rare_cost_prop.std()  

In [25]:
z_score = relative_rare_cost_prop / standard_errors
z_score = pd.DataFrame(z_score.sort_values(ascending = False))    

z_score.reset_index(inplace = True)

In [26]:
z_score.columns = ['practice', 'z_score']

In [27]:
fin = (practices.groupby(['code'])[['code', 'name']]).head()

In [28]:
result = z_score.merge(fin, how = 'left', left_on = 'practice',right_on = 'code').drop('code', axis = 1)

In [29]:
df = result.groupby('practice').first().sort_values('z_score', ascending = False).reset_index()[:100]

In [30]:
rare_scripts = list(zip(df.practice, df.name, df.z_score))

In [31]:
grader.score.dw__rare_scripts(rare_scripts)

Your score: 0.930


*Copyright &copy; 2021 WorldQuant University. This content is licensed solely for personal use. Redistribution or publication of this material is strictly prohibited.*